To-do: 

(1) Make sure your basemap and layers are in the same *projected* coordinate system. Change the basemap in Map Properties > Coordinate Systems and change layers with the Project tool. 

(2) Replace the generalized parameters in the following cell with your paths and layer names. See "Notes" below for more info on naming conventions. 

(3) Click "Run all cells" (two triangles) in the ribbon above this cell. 

Notes: 
- Use double quotation marks for any paths, layers, or field names (i.e., strings) - but not for numeric entries.
- Keep the r in front of your gdb_path string - this makes it readable. 
- If a layer you want to use is in a group, make sure to include it's group name in it's path (i.e., "GroupName/LayerName" - the relative path.)
- For entries in "buffer_features", the buffer distance must be formatted as "X Mile"
- If you have more buffer features than outlined here, you can just add them to the list!
- Outputs of interest: (1) any of the PhiH_poly_masked layers (polygons of buffered PhiH with cutoffs of 80 & 200), (2) any of the PhiH_masked layers (same, but rasters), and (3) PhiH_Total_Counties (table containing square mileage calculations in the order of PhiH>0, PhiH>80, and PhiH>200). 

In [ ]:
# Absolute path to your geodatabase 
gdb_path = r"C:\absolute\path\to\your\.gdb"

# Relative path to layer defining the spatial extent of your analysis (e.g., state or basin boundary)
extent = "GroupName/LayerName"

# Relative path to layer defining county boundaries and the field that contains county names 
counties = "GroupName/LayerName" 
county_field = "FieldName" 

# Relative path to layer of basement deeper than 4,000 ft
basement = "LayerName" 

# Relative path to thickness and porosity rasters - keep the Raster(), just replace LayerName 
thickness = Raster("LayerName")
porosity = Raster("LayerName")

# List of features to buffer and remove. Format (layerName, "X Mile") 
buffer_features = [ 
    ("LayerName", "X Mile"), 
    ("LayerName", "X Mile"), 
    ("LayerName", "X Mile"), 
    ("LayerName", "X Mile"), 
    ("LayerName", "X Mile"), 
]

In [ ]:
##### SETUP ##### 

import os 
import arcpy 
from arcpy.sa import * 

arcpy.env.workspace = gdb_path 
arcpy.env.overwriteOutput = True
arcpy.env.mask = extent
arcpy.env.extent = extent 

project = arcpy.mp.ArcGISProject("CURRENT") 
map_obj = project.activeMap

print("Environment set-up complete. \n")

##### FUNCTIONS #####

def get_layer_by_name(layer_name):
    """Finds layers by name, so they can be in groups"""
    for lyr in map_obj.listLayers():
        if lyr.isGroupLayer:
            for sub_lyr in lyr.listLayers():
                if sub_lyr.name == os.path.basename(layer_name):
                    return sub_lyr
        elif lyr.name == os.path.basename(layer_name):
            return lyr
    return None

def check_data_exists(name):
    """Checks if a dataset/layer exists"""
    if arcpy.Exists(name):
        return True
    return get_layer_by_name(name) is not None

def apply_buffer_and_mask(input_feature, buffer_name, distance, PhiH_raster):
    """Buffers the input feature and extracts the raster outside of that buffer"""
    try:
        arcpy.analysis.Buffer(input_feature, buffer_name, distance)
        return ExtractByMask(PhiH_raster, buffer_name, "OUTSIDE")
    except Exception as e:
        print(f"\nError buffering {input_feature}: {e}")
        return PhiH_raster  # Returns unchanged raster if buffering fails - continues on??? 

def tabulate_intersection(zone_fc, zone_field, class_fc, output_table):
    """Calculates square mileage of feasibility in each county"""
    try:
        arcpy.analysis.TabulateIntersection(
            in_zone_features=zone_fc,
            zone_fields=zone_field,
            in_class_features=class_fc,
            out_table=output_table,
            class_fields="",
            sum_fields="",
            xy_tolerance="",
            out_units="SQUARE_MILES_US"
        )
        print(f"\nCalculated {output_table}")
    except Exception as e:
        print(f"\nError calculating {output_table}: {e}")

def find_layers_by_name(map_layers, name_to_find):
    """Finds layers on the map"""
    for lyr in map_layers:
        if lyr.name == name_to_find:
            yield lyr
        elif lyr.isGroupLayer:
            yield from find_layers_by_name(lyr.listLayers(), name_to_find)

##### BEGIN ANALYSES ##### 

print("Checking if required inputs are present. \n")

inputs_to_check = {
    "GeoDatabase": gdb_path,
    "Boundary": extent,
    "Basement": basement,
    "Counties": counties
}

for feature_name, _ in buffer_features:
    inputs_to_check[f"Buffer: {feature_name}"] = feature_name
    
all_valid = True
for label, path in inputs_to_check.items():
    exists = check_data_exists(path)
    status = "Found" if exists else "MISSING"
    print(f"{status:<10} | {label}")
    if not exists:
        all_valid = False

if not all_valid:
    raise Exception("Stopping script -- missing required inputs. \n")

print("\nCalculating PhiH")

try:
    PhiH = thickness * porosity
    PhiH_raster = PhiH
    print("PhiH calculated\n")
except Exception as e:
    raise RuntimeError(f"Failed to calculate PhiH: {e}")

for feature_name, distance in buffer_features:
    safe_name = os.path.basename(feature_name).replace(" ", "_")
    buffer_output_name = f"{safe_name}_Buffer"
    input_layer = get_layer_by_name(feature_name) or feature_name
    print(f"Buffering {feature_name} with distance {distance}")
    PhiH_raster = apply_buffer_and_mask(input_layer, buffer_output_name, distance, PhiH_raster)

try:
    PhiH_masked = ExtractByMask(PhiH_raster, basement, "INSIDE")
    PhiH_masked_80 = Con(PhiH_masked > 80, PhiH_masked)
    PhiH_masked_200 = Con(PhiH_masked > 200, PhiH_masked)
    print( "\nPhiH thresholds created (80 & 200).")
except Exception as e:
    raise RuntimeError(f"Failed during final masking: {e}")

ZonalStatisticsAsTable(counties, county_field, PhiH_masked, "CountyMeans", "DATA", "MEAN")

arcpy.conversion.RasterToPolygon(Int(PhiH_masked), "PhiH_poly_masked", "NO_SIMPLIFY")
arcpy.conversion.RasterToPolygon(Int(PhiH_masked_80), "PhiH_80_poly_masked", "NO_SIMPLIFY")
arcpy.conversion.RasterToPolygon(Int(PhiH_masked_200), "PhiH_200_poly_masked", "NO_SIMPLIFY")

tabulate_intersection(counties, county_field, "PhiH_poly_masked", "PhiH_Total_Counties")
tabulate_intersection(counties, county_field, "PhiH_80_poly_masked", "PhiH_80_Counties")
tabulate_intersection(counties, county_field, "PhiH_200_poly_masked", "PhiH_200_Counties")

join_1 = arcpy.management.AddJoin("PhiH_Total_Counties", county_field, 
                                            "PhiH_80_Counties", county_field)
join_2 = arcpy.management.AddJoin("PhiH_Total_Counties", county_field, 
                                            "PhiH_200_Counties", county_field)

print("Joined county ranking tables \n")

print("All done!")